# Example with dummy data

This notebook shows a simplified use case utilizing a dummy set of randomly generated particle data. It is by default set to simulate a catalog of stars with Gaia DR2 astrometry & photometry, but you may change the parameters defined below to your liking. In short, the pipeline is accessible via the class `Ananke`, which method `run` can eventually be called after creation of an `Ananke` object to run the full ananke pipeline. The method `run` is designed to returns the catalog in the form of an `Output` object of the submodule `galaxia_ananke` which can be used as a `vaex` dataframe.

Below are the packages that this notebook must import:

In [1]:
import numpy as np
import ananke as an
an.__version__

'0.3.0+2.g7efc7c0'

We define here some dummy input data. `Ananke` has a method `make_dummy_particles_input` to produce such data in a dictionary formatted output. Below we show how to call it and the keys that compose that dictionary.

In [2]:
np.random.seed(0)
p = an.Ananke.make_dummy_particles_input()
p.keys()

dict_keys(['pos3', 'vel3', 'mass', 'age', 'feh', 'helium', 'carbon', 'nitrogen', 'oxygen', 'neon', 'magnesium', 'silicon', 'sulphur', 'calcium', 'alpha', 'parentid', 'partitionid', 'dform', 'id', 'log10_NH'])

The input data must be formatted as a dictionary of equal-length arrays. The dictionary must have the following entries:
- key `pos3`: particle position coordinates in $kpc$ (shape Nx3)
- key `vel3`: particle velocity coordinates in $km.s^{-1}$ (shape Nx3)
- key `mass`: particle stellar mass in solar masses
- key `age`: particle log10 stellar age in years
- key `feh`: particle stellar metallicity \[Fe/H\] in dex relative to solar

Additionally, the following entries can optionally be added:
- key `parentid`: index to give to the parent particle
- key `id`: additional index to classify the parent particle
- key `log10_NH`: log10 hydrogen column densities between Observer position and particle in $cm^{-2}$ - must be provided to estimate extinctions
- key `dform`: particle formation distance
- keys `helium`, `carbon`, `nitrogen`, `oxygen`, `neon`, `magnesium`, `silicon`, `sulphur`, `calcium`: particle various chemical abundances \[X/H\]
- key `alpha`: particle alpha chemical abundances \[Mg/Fe\]

Ananke will compute the phase space densities that are used to determine particle smoothing lengths, but one can include pre-computed densities with the following entries:
- key `rho_pos`: particle density in position space in $kpc^{-3}$
- key `rho_vel`: particle density in velocity space in $km^{-3}.s^{3}$

At any time, you may access this format via the helper of the `Ananke.make_dummy_particles_input` method using

`help(an.Ananke.make_dummy_particles_input)`

We define below some parameters for Ananke such as
 - the observer position `observer`
 - the shell of particles to mask `rshell`
 - the sampling factor `fsample`
 - the photometric system of choice `photo_sys` (in our case Gaia DR2)
 - the CMD `cmd_magnames` and its box limits `cmd_box`

In [3]:
D = 200 # *units.kpc

observer = np.nan*np.ones(3)
while not np.linalg.norm(observer)<1:
    observer = 2*np.random.rand(3)-1

observer *= D/np.linalg.norm(observer)

rshell = [0, 2*D]

fsample = 0.01

photo_sys = 'padova/GAIA__DR2'

cmd_magnames = {'magnitude': 'G',
                'color_minuend': 'Gbp',
                'color_subtrahend': 'Grp'}

cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

For more details regarding these parameters and more, you may consult the docstring associated to the class `Ananke` via the lines:

`help(an.Ananke.__init__)`

or

`help(an.Ananke)`

Note that among those are 3 special parameters, `d_params`, `e_params` and `err_params`, which receive dictionaries of subparameters to configure the sub-steps of the pipeline corresponding respectively to the phase space density estimation, the extinction mapping and the instrument error modeling. Each have further documentation that can be accessed calling the following methods:

`an.display_density_docs()` for `d_params`

`an.display_extinction_docs()` for `e_params`

`an.display_errormodel_docs()` for `err_params`

We show below how to create the `Ananke` pipeline object before calling `run`, using the parameters we defined above.

In [4]:
name = 'sim'
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

/home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/ananke/utils.py:31: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/ananke/utils.py:31: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()


As mentioned above, the `Ananke` method `run` runs the pipeline once it has been created. Some additional parameters can be provided to this method, which are listed in its docstring (accessible by calling `help(ananke.run)`). Those include:
- `input_dir`, `output_dir`, `i_o_dir` - Optional arguments to specify paths for the directories where `ananke` should generate input and output data. If the `i_o_dir` keyword argument is provided, it overrides any path given to the `input_dir` and `output_dir` keyword arguments.
- `k_factor` – Scaling factor applied to the kernels lengths to adjust all the kernels sizes uniformly. Lower values reduces the kernels extents, while higher values increases them. Default to 1 (no adjustment).
- `surveyname` – Optional name `Galaxia` should use for the output files. Default to `'survey'`.
- `n_jobs` – Number of independent catalog generations ran in parallel. Default to 1.
- `verbose` – Verbose boolean flag to allow pipeline to print what it's doing to stdout. Default to `True`.
- `rand_seed` – Seed to be used by `Galaxia`'s pseudorandom number generator. Default to 17052
- `nstart` – Index at which to start indexing synthetic stars. Default to 0
For our example however, we will use the method as it is.

<!-- > [!WARNING]  
> Some of the `run` method parameters are currently not properly implemented, using them may lead to unexpected results -->

<div class="admonition warning">
<div class="title">Warning</div>
<p>Some of the `run` method parameters are currently not properly implemented, using them may lead to unexpected results</p>
</div>

In [5]:
survey = ananke.run()

Dimensions = 3
Normalization constant of Kernel type 3: 0.596831
Reading ASCII format file: to_enbid 
Read 300000 records
Allocated 3.8147 MByte for particle storage.
Type = 1 Particles = 100000
Total particles = 100000
Allocated 22.1252 MByte for Binary-Tree and Entropy bins.

Scaling Co-ordinates as x[i]=x[i]/h[i] with h[i]->
1 1 1 

Starting to Build Tree .......
Particle Type = 1 First node number = 1
                  Last node number  = 199999
Total number of nodes = 199999
Treebuild time = 0.163068 s 

Density Calculation. Smoothing .....
Evaluated =   1 % Time Left = 1.176009 s of 1.187900 s Par no = 96805 Density = 4.540411e-02 
Evaluated =   2 % Time Left = 1.144187 s of 1.167550 s Par no = 55454 Density = 4.964532e-03 
Evaluated =   3 % Time Left = 1.165184 s of 1.201233 s Par no = 27717 Density = 1.827217e-01 
Evaluated =   4 % Time Left = 1.182252 s of 1.231525 s Par no = 42234 Density = 2.712376e-02 
Evaluated =   5 % Time Left = 1.180173 s of 1.242300 s Par no = 99785 De

/home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/enbid_ananke/__init__.py:271: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  usedvals = pd.read_table(path / CONSTANTS.usedvalues, header=None, delim_whitespace=True,


Dimensions = 3
Normalization constant of Kernel type 3: 0.596831
Reading ASCII format file: to_enbid 
Read 300000 records
Allocated 3.8147 MByte for particle storage.
Type = 1 Particles = 100000
Total particles = 100000
Allocated 22.1252 MByte for Binary-Tree and Entropy bins.

Scaling Co-ordinates as x[i]=x[i]/h[i] with h[i]->
1 1 1 

Starting to Build Tree .......
Particle Type = 1 First node number = 1
                  Last node number  = 199999
Total number of nodes = 199999
Treebuild time = 0.159468 s 

Density Calculation. Smoothing .....
Evaluated =   1 % Time Left = 1.214124 s of 1.226400 s Par no = 3379 Density = 1.586355e-02 
Evaluated =   2 % Time Left = 1.274183 s of 1.300200 s Par no = 75725 Density = 4.756319e-02 
Evaluated =   3 % Time Left = 1.311782 s of 1.352367 s Par no = 95160 Density = 6.928090e-03 
Evaluated =   4 % Time Left = 1.261523 s of 1.314100 s Par no = 40983 Density = 2.235913e-02 
Evaluated =   5 % Time Left = 1.259326 s of 1.325620 s Par no = 79587 Den

/home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/enbid_ananke/__init__.py:271: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  usedvals = pd.read_table(path / CONSTANTS.usedvalues, header=None, delim_whitespace=True,


JOB 1/1 | Isochrone Grid Size:                (Age bins=71,Feh bins=34,Alpha bins=1)
JOB 1/1 | Time Isochrone Reading              3.62717     
JOB 1/1 | ------------------------------
JOB 1/1 | nbody1/sim/sim.ebf  Sat No=0
JOB 1/1 | Particles=100000
JOB 1/1 | Satellite Info
JOB 1/1 | Particles=100000 Mass=5.49989e+08 0.379706
JOB 1/1 | Total Stars=2213240 accepted=1611712 rejected=601528
JOB 1/1 | -----------Done---------------
JOB 1/1 | Total stars written                 1611712                 
JOB 1/1 | 31
JOB 1/1 | File written-                       /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.ebf
JOB 1/1 | Calculating magnitudes in GAIA__DR2 system................
JOB 1/1 | initializing isochrone data
JOB 1/1 | interpolating on isochrone tables
JOB 1/1 | Total Time=                         8.9051      
Exported the following quantities from /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.ebf to /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.h5 for

...

/home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/ananke/utils.py:31: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/ananke/ErrorModelDriver.py:122: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  i_max_err = np.abs(df[prop_err_name] if isinstance(df, pd.DataFrame) else df[prop_err_name].to_pandas_series()).argmax()


Exported the following quantities to /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.h5
['vr_Err', 'pi_Sig', 'ra_Sig', 'vr_Sig', 'dec_Err', 'mura_Sig', 'gaia__dr2_g_Sig', 'gaia__dr2_gbp_Err', 'mudec_Sig', 'gaia__dr2_gbp_Sig', 'pi_Err', 'gaia__dr2_grp_Err', 'mura_Err', 'mudec_Err', 'mudec_Clean', 'mura_Clean', 'ra_Err', 'pi_Clean', 'dec_Sig', 'ra_Clean', 'dec_Clean', 'vr_Clean', 'gaia__dr2_g_Err', 'gaia__dr2_grp_Sig']
Overwritten the following quantities to /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.h5
['pi', 'mura', 'gaia__dr2_gbp', 'dec', 'vr', 'gaia__dr2_grp', 'ra', 'mudec', 'gaia__dr2_g']


[06/20/25 13:02:13] WARNING  could not close memmap for column                                   ]8;id=694204;file:///home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/vaex/dataset_mmap.py\dataset_mmap.py]8;;\:]8;id=73207;file:///home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/vaex/dataset_mmap.py#94\94]8;;\
                             /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.h5                           

Running convert_icrs_to_galactic post-processing pipeline
Overwritten the following quantities to /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.h5
['glon', 'mub', 'glat', 'mul']


[06/20/25 13:02:15] WARNING  could not close memmap for column                                   ]8;id=3877;file:///home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/vaex/dataset_mmap.py\dataset_mmap.py]8;;\:]8;id=504863;file:///home/athob/miniconda3/envs/test_pypi/lib/python3.10/site-packages/vaex/dataset_mmap.py#94\94]8;;\
                             /home/athob/pypi_sand_box/py-ananke/jupyter/survey.sim.0.h5                           

The output is saved as a `vaex` dataframe, with its columns organized in alphabetical order. The photometry is stored in columns that are named according to their associated photometric system and filter name. The keys are in the lowercase format `photosys_filtername` where `photo_sys` is the photometric system and `filtername` is the filter name. In this example, the apparent photometry in filters `gbp`, `grp` & `g` of the Gaia DR2 system (identified as `GAIA__DR2` in Galaxia) are respectively under keys `gaia__dr2_gbp`, `gaia__dr2_grp` & `gaia__dr2_g`. Beside these, the columns include:

- key `A_0` for the reference extinction which extinction coefficients are based on (at $\lambda = 550 \, nm$ in the case of Gaia DR2)
- key `A_{filter_name}` for the extinction value in each filter designated by `filter_name` (here `A_gaia__dr2_g`, `A_gaia__dr2_gbp` & `A_gaia__dr2_grp`)
- key `E(B-V)` for the reddening index
- key `age` for the $log_{10}$ stellar age in years
- key `alpha`, `calcium`, `carbon`, `helium`, `magnesium`, `neon`, `nitrogen`, `oxygen`, `silicon`, `sulphur` for the various chemical abundances as given as input
- key `dec`, `ra` for the astrometric declination and right ascension celestial coordinates in degrees
- key `dform` for the formation distance as given as input
- key `dmod` for the distance modulus
- key `feh` for the stellar metallicity \[Fe/H\] in dex relative to solar
- key `{filter_name}_Intrinsic` for the intrinsic stellar photometry in each filter designated by `filter_name` (here `gaia__dr2_g_Intrinsic`, `gaia__dr2_gbp_Intrinsic` & `gaia__dr2_grp_Intrinsic`)
- key `glat`, `glon` for the astrometric galactic latitude and longitude celestrial coordinates in degrees
- key `grav` for the $log_{10}$ surface gravity in CGS units
- key `log10_NH` for the $log_{10}$ hydrogen column density between Observer position and star in $cm^{-2}$
- key `lum` for the stellar luminosity in solar luminosities
- key `mact`, `mtip`, `smass` for respectively the current stellar mass, the mass of that same star at tip of giant branch for its given age \& metallicity and its stellar mass on zero-age main sequence, all in solar masses
- key `mub`, `mudec`, `mul`, `mura` for the astrometric proper motions, respectively in the direction of the galactic latitude, declination, galactic longitude and right ascension, all in milliarcseconds per year
- key `parentid` for the parent particle index as given as input
- key `partid` for the flag that identifies stars that are *not* central relatively to their parent particle
- key `pi` for the star parallax in milliarcseconds
- key `px`, `py`, `pz` for the star position cartesian coordinates in $kpc$ relative to the Observer's position
- key `rad` for the star distance to the Observer in $kpc$
- key `satid` for the index ``id`` the population the parent particle is part of
- key `teff` for the star effective temperature in Kelvin
- key `vr` for the star astrometric radial velocity in $km.s^{-1}$
- key `vx`, `vy`, `vz` for the star velocity cartesian coordinates in $km.s^{-1}$ relative to the Observer's velocity

Additionally, astrometric and photometric quantities `X` all have associated columns identified as:
- key `{X}_Sig` for the standard error on the quantity `X`
- key `{X}_Err` for the actual drawn gaussian error on the quantity `X`

Taking the photometry as an example of the latter, the standard errors are stored under `gaia__dr2_g_Sig`, `gaia__dr2_gbp_Sig` & `gaia__dr2_grp_Sig`, while the drawn errors are under `gaia__dr2_g_Err`, `gaia__dr2_gbp_Err` & `gaia__dr2_grp_Err`.

In [6]:
survey

#,A_0,A_gaia__dr2_g,A_gaia__dr2_gbp,A_gaia__dr2_grp,E(B-V),age,alpha,calcium,carbon,dec,dec_Clean,dec_Err,dec_Sig,dform,dmod,feh,gaia__dr2_g,gaia__dr2_g_Err,gaia__dr2_g_Intrinsic,gaia__dr2_g_Sig,gaia__dr2_gbp,gaia__dr2_gbp_Err,gaia__dr2_gbp_Intrinsic,gaia__dr2_gbp_Sig,gaia__dr2_grp,gaia__dr2_grp_Err,gaia__dr2_grp_Intrinsic,gaia__dr2_grp_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Clean,mudec_Err,mudec_Sig,mul,mura,mura_Clean,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,partitionid,pi,pi_Clean,pi_Err,pi_Sig,px,py,pz,ra,ra_Clean,ra_Err,ra_Sig,rad,satid,silicon,smass,sulphur,teff,vr,vr_Clean,vr_Err,vr_Sig,vx,vy,vz
0,0.7093977911594089,0.5220698544539614,0.709193133907145,0.40556140505219107,0.22883799714819641,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,43.89190561692206,nan,nan,0.0,20.87123488088669,-0.8867433071136475,nan,nan,7.8221183,nan,nan,nan,8.470699,nan,nan,nan,7.0639534,nan,nan,nan,4.769691,-0.7315406799316406,5.72094992870491e+21,0.071013965,0.50516033,-0.5486690402030945,0.886202,nan,nan,-0.2413280326361944,nan,nan,nan,nan,0.29630942610455613,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,0,0,nan,0.006695037652823929,nan,nan,-142.03525597960677,46.20841677042607,0.692591828190988,nan,73.8027984762779,nan,nan,149.36435788052748,0,-0.7563512325286865,0.5054699889829319,-1.0924330949783325,4285.4614,-156.7385192430598,-156.7385192430598,nan,nan,67.38685760498046,-300.3354850769043,55.13861343383789
1,0.7566822063230371,0.5514594111171848,0.7534424991156238,0.4311956707179316,0.2440910342977539,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,43.73518629916732,nan,nan,0.0,20.87527071994942,-0.8867433071136475,nan,nan,7.993824,nan,nan,nan,8.668457,nan,nan,nan,7.2174444,nan,nan,nan,4.7857795,-0.7315406799316406,6.102275857443847e+21,0.06093182,0.48871416,-0.5486690402030945,0.886202,nan,nan,-0.24008076703265266,nan,nan,nan,nan,0.2913066369253815,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,0,nan,0.006682605995616206,nan,nan,-142.2608617629229,46.417971376138425,-0.11534282804241303,nan,73.42607530130886,nan,nan,149.64222051337467,0,-0.7563512325286865,0.48905900859965035,-1.0924330949783325,4198.5107,-156.97767104891037,-156.97767104891037,nan,nan,67.7484200428451,-298.29957314526564,52.69162146715175
2,136.64647824055086,2904.2323916733826,1370.444979191365,78.02675978260831,44.07950910985512,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,44.364568278584706,nan,nan,0.0,20.888157433594138,-0.8867433071136475,nan,nan,8.71385,nan,nan,nan,9.517461,nan,nan,nan,7.8597684,nan,nan,nan,4.8554068,-0.7315406799316406,1.101987727746378e+24,0.03486588,0.42435014,-0.5486690402030945,0.886202,nan,nan,-0.23863717453488734,nan,nan,nan,nan,0.2958006234265731,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,0,nan,0.006643065174432655,nan,nan,-142.9270461828407,47.20842184009032,1.8396875936103703,nan,74.04989339204981,nan,nan,150.5329202321734,0,-0.7563512325286865,0.4250665825385847,-1.0924330949783325,3935.5205,-158.4535061010785,-158.4535061010785,nan,nan,68.03093236871236,-301.4758806839234,56.09200627357448
3,0.7150916084000247,0.59382870248778,0.7598827721299756,0.4223180357513091,0.23067471238710474,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,43.89621033939353,nan,nan,0.0,20.86868071849355,-0.8867433071136475,nan,nan,5.1929526,nan,nan,nan,5.4745564,nan,nan,nan,4.7391353,nan,nan,nan,4.5336547,-0.7315406799316406,5.766867809677618e+21,0.6409676,0.74067265,-0.5486690402030945,0.886202,nan,nan,-0.24429902665121836,nan,nan,nan,nan,0.2940592689423018,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,0,nan,0.006702917225320306,nan,nan,-141.88055546200977,46.1152954631301,0.7600375956495637,nan,73.8447277872187,nan,nan,149.18877354213694,0,-0.7563512325286865,0.740707

Please refer to [`vaex`'s documentation](https://vaex.io/docs/tutorial.html) for further help on how to use `vaex` dataframes: the following line for example isolate only the rows with non-NaN photometry.

In [7]:
survey[~survey.gaia__dr2_g.isna()]

#,A_0,A_gaia__dr2_g,A_gaia__dr2_gbp,A_gaia__dr2_grp,E(B-V),age,alpha,calcium,carbon,dec,dec_Clean,dec_Err,dec_Sig,dform,dmod,feh,gaia__dr2_g,gaia__dr2_g_Err,gaia__dr2_g_Intrinsic,gaia__dr2_g_Sig,gaia__dr2_gbp,gaia__dr2_gbp_Err,gaia__dr2_gbp_Intrinsic,gaia__dr2_gbp_Sig,gaia__dr2_grp,gaia__dr2_grp_Err,gaia__dr2_grp_Intrinsic,gaia__dr2_grp_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Clean,mudec_Err,mudec_Sig,mul,mura,mura_Clean,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,partitionid,pi,pi_Clean,pi_Err,pi_Sig,px,py,pz,ra,ra_Clean,ra_Err,ra_Sig,rad,satid,silicon,smass,sulphur,teff,vr,vr_Clean,vr_Err,vr_Sig,vx,vy,vz
0,0.35884632402023736,0.2624818507694532,0.35801790759137203,0.20466172853140513,0.11575687871620559,9.692870140075684,0.3099161982536316,-1.1548902988433838,-0.4665657579898834,18.868015796305663,18.868015778565187,1.7740477296505672e-08,1.691274e-07,0.0,21.118883008502078,-0.48292723298072815,19.868202,0.012658170630218355,-1.5258217,0.009227399,20.504751,-0.16132866071418359,-0.8108196,0.17850117,18.7721,-0.22652792657603063,-2.3249161,0.17850117,-14.991976836287702,-177.95690294976794,1.2548169,-0.27809077501296997,2.89392196790514e+21,448.28244,1.1287807,-0.17301106452941895,1.1379396,-0.15521054578348675,-1.3589967238651148,-0.3270482743773745,-1.0319484494877402,1.0656426,1.5702165708100968,0.8017470282407725,0.1632684267596954,0.638478601481077,1.0656426,-0.7034189105033875,-0.09953558444976807,-0.4723181426525116,49,1,0,-0.6145466549364363,0.0059734247671244375,-0.6205200797035608,0.60885864,-161.6071229139394,-5.765154922998849,-43.30577433967765,74.02626514013622,74.02626502725049,1.1288572264651892e-07,1.691274e-07,167.40815176975815,0,-0.6018809676170349,1.1408413190663447,-0.7073405385017395,4129.926,69.84271348583249,69.84271348583249,nan,nan,-44.649975087957735,-287.75369665394226,-65.06155285732873
1,0.5999316419840474,0.3951491437418692,0.5843014320686067,0.32756794644856657,0.19352633612388623,9.988179206848145,0.11780641227960587,0.09089594334363937,-0.26741287112236023,28.32392700064621,28.323927151103437,-1.5045722692676327e-07,7.903525e-08,0.0,21.313550824736375,-0.6772944927215576,19.078148,0.007681412416285324,-2.6382327,0.005971381,20.259933,-0.07032803519170927,-1.5675896,0.09647448,17.950785,-0.07180883013983862,-3.6185248,0.09647448,-18.536442761608956,165.70363972881057,0.3074857,-0.21355026960372925,4.838158403097156e+21,1852.9218,0.85137147,-0.55948805809021,0.91256934,0.3131789409812662,-0.10381565378748775,-0.14986186911488497,0.04604621532739723,0.55013824,0.48286076265325345,0.5660899886174509,0.1728590301091314,0.39323095850831946,0.55013824,-0.3997773826122284,-0.7002913355827332,-0.08155464380979538,144,1,0,-0.15274891158827747,0.005461222050076331,-0.1582101336383538,0.2845269,-168.233346583228,42.8707615557611,-58.21183715231835,59.864629221697925,59.86462915929865,6.239927571995072e-08,7.903525e-08,183.10919988796704,0,-0.3103104829788208,0.914106032084624,-0.38073936104774475,3662.81,-25.762507382985724,-25.762507382985724,nan,nan,-27.923177997805908,-197.62939339603813,16.189820511232778
2,0.17608195267994303,0.1377033942738126,0.1808537019520944,0.10255663320845036,0.056800629896755814,9.579007148742676,-0.6709350943565369,-0.9936923980712891,-0.9404328465461731,33.04906634892174,33.04906634871992,2.0182171922849153e-10,3.6569543e-07,0.0,21.38615087842112,-0.9346992373466492,20.56803,-0.03600744374280186,-0.9198183,0.014033196,21.098215,-0.07126815067421044,-0.3975206,0.32202673,19.32279,-0.5803682522135298,-1.585548,0.32202673,-8.484585766867523,168.82659578503788,1.813334,-0.15246030688285828,1.4200157474188954e+21,204.61284,1.146,-1.605634331703186,1.1506394,-0.33249713324821484,-1.1621686215766727,-0.12728573316172526,-1.0348828884149475,2.0831733,1.2421141410353262,0.550241745054646,0.13419926585904626,0.4160424791955997,2.0831733,-0.9467113018035889,-0.7777135372161865,-0.36034896969795227,355,1,0,-2.8237769876652146,0.0052816525